<a href="https://colab.research.google.com/github/mahesh-keswani/ML-DL-Basics/blob/main/keras_examples/GreatExampleOfSequentialAndFunctionalAPIKeras.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Resource: https://www.youtube.com/watch?v=pAhPiF3yiXI

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.datasets import mnist

In [2]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()

11493376/11490434 [==============================] - 0s 0us/step


In [3]:
print(x_train.shape, y_train.shape)

(60000, 28, 28) (60000,)


In [4]:
# there are 60000 images of shape 28x28

In [5]:
# converting to float32 for reducing computational cost
x_train = x_train.reshape(-1, 28*28).astype("float32") / 255.0
x_test = x_test.reshape(-1, 28*28).astype("float32") / 255.0

<h1>Sequential API</h1>

In [6]:
# Sequential API ( It is very convenient and easy to use when you have one input and one output )
model = keras.Sequential()
# when you don't specify input shape, during fit it will interpret the shape by itself, but only after the fit then only you can do model.summary()
model.add( layers.Dense( units = 128, activation='relu' ) )
model.add( layers.Dense( units = 64, activation='relu' ) )
model.add( layers.Dense( units = 10) )

# when you don't specify softmax, you can do this in loss function as well

In [8]:
# if we have labels as integers then we use Sparse, but if labels are one hot encode we can simply use CategoricalCrossEntropy
# from_logits = True, then it will use softmax first then calculate loss

model.compile(
    loss = keras.losses.SparseCategoricalCrossentropy(from_logits = True),
    optimizer = keras.optimizers.Adam(),
    metrics = ['accuracy']
)

In [ ]:
# you cannot use model.summary() here, it will result in error, only after fit

In [9]:
model.fit(x_train, y_train, epochs = 3, verbose = 2, batch_size = 32)

Epoch 1/3
1875/1875 - 4s - loss: 0.2385 - accuracy: 0.9296
Epoch 2/3
1875/1875 - 3s - loss: 0.1034 - accuracy: 0.9681
Epoch 3/3
1875/1875 - 3s - loss: 0.0721 - accuracy: 0.9775


In [11]:
model.evaluate(x_test, y_test, batch_size=32, verbose=2)

313/313 - 0s - loss: 0.0909 - accuracy: 0.9731


[0.09085403382778168, 0.9731000065803528]

<h1>Functional API</h1>

In [12]:
# Now Functional API ( which is bit more flexible )
# creating same network as above, but with functional API

# can also provide additional argument 'name' which can be useful for debugging
inputs = layers.Input( shape = (784) )
x = layers.Dense( units = 128, activation = 'relu' )(inputs)
x = layers.Dense( units = 64, activation = 'relu' )(x)
outputs = layers.Dense( units = 10 )(x)


In [13]:
model = keras.Model( inputs = inputs, outputs = outputs )

In [14]:
model.compile(
    loss = keras.losses.SparseCategoricalCrossentropy(from_logits = True),
    optimizer = keras.optimizers.Adam(),
    metrics = ['accuracy']
)

In [15]:
model.fit(x_train, y_train, epochs = 3, verbose = 2, batch_size = 32)

Epoch 1/3
1875/1875 - 3s - loss: 0.2366 - accuracy: 0.9312
Epoch 2/3
1875/1875 - 3s - loss: 0.0999 - accuracy: 0.9697
Epoch 3/3
1875/1875 - 3s - loss: 0.0698 - accuracy: 0.9782


In [17]:
model.evaluate( x_test, y_test, batch_size=32, verbose=2 )

313/313 - 0s - loss: 0.0914 - accuracy: 0.9717


[0.09143621474504471, 0.9717000126838684]

<h1>Getting output of intermediate layers</h1>

In [19]:
# If you want the output of intermediate layer

# printing the output of every layer
input_layer = x_train
for layer in model.layers:
    output = layer(input_layer)
    print( output.shape )
    input_layer = output

(60000, 784)
(60000, 128)
(60000, 64)
(60000, 10)


<h1>Getting prediction from intermediate layer</h1>

In [21]:
# using the previous model inputs and using the second last layer for output
# if it had name property, say name = 'second_last_layer', then we can also do, model.get_layer(name = 'second_last_layer').output
model = keras.Model( inputs = model.inputs, outputs = model.layers[-2].output )
latent_representation = model.predict( x_train )